# NGCF
## Introduction
Collaborative filtering is one of the oldest and most explored methods in recommendation. It created a connection between users, who interact with the system such as users of a video streaming system and items, which is what a system offers such as videos in a video streaming service. Before graphs, user-item relationship was modeled using sparse matrices, where the user-item relationship was represented as a rating score. For instance, a user can give one to five star to a movie. The task of collaborative filtering was to use the known ratings, and predict ratings for unknown rarings.

![images/mf.png](images/mf.png)

The above image represnts 10 users and 100 items with density of 0.05. The colour of the squares is indicative of rating score. The lightest gray is 1 and black is 5, whicle white means there is not rating for an item by a uer. Even in this image we can see that the the matrix is mostly filled with empty space. In real world scenarios, we often have 
tens of millions of users and tens of millions of items with densities around 0.005.' [movielens20M dataset](https://grouplens.org/datasets/movielens/20m/). Other larger datasets could have potentially much lower densities. Imagine that a music straming service such as Amazon Music has 100000000 songs. If an average person rates 200 songs, the density will be at 0.000002 and 99.9998 percent of the rating matrix will be occupied by empty space.



Useful as all variations of collaborative filtering were, they suffered from a series of deficinecies, chief amongst them were handling of very large sparse matrices and loss of collaborative signal.

We have already seen the root-cause of sparsity. The Loss of collaborative signal results from the fact that users and items pass through separate embedding layers, thus embedding does not capture user-item interactions. 

Neural Graph collaborative filtering addresses both issues through modeling user-item relationshsips as a biprtite graph with rating represented as weight of a collaboration.
![bipartite](images/bipartiteCF.png)

## The NGCF Paradigm
There are three steps to implement NGCF, 1) Loading the data into a bipartite graph. 2) Embedding layers that implment first-order embedding capturing firt-order interaction, and higher-order embedding that captures higher-order interactions for constructing more accurate informatio, 3) prediction, which proposes user-item possible interactions using embedding.

### Data Loading
[Amazon review dataset](https://cseweb.ucsd.edu/~jmcauley/datasets/amazon_v2/) can be summarized to uers, items, and ratings. Loading the data involves creating a class that includes users, items, and the link between the two.
```python
class Data(object):
    def __init__(self, path, batch_size):
        ...
        
        user_item_src = []
        user_item_dst = []
        
        ...
```
The important part to notice here is two arrays that will include the interactions.
The dataset is broken into trainingand test sets. Each line has user id as first entry and a list of items with which the user interacts.

For instance, in the Amazon dataset the fist line is as follows:
```
0 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52
```
This means that user 0 interacts with items 0, 1, 2, ..., 51, 52. There are a total of 

We need to create `user_item_src` to have a length of 52 with all entries containing the `user_id==0`. The `user_item_dst` will have the same length and contains all the item ids listed in the above line.

We now have two arrays:
`user_item_src = [0, 0, ..., 0]` with `len(user_item_src)==52` and `user_item_dst = [0, 1, ..., 51, 52]` with `len(user_item_dst)==52`. 

We then repeast the process for all the users in the system. We can then use each pair of correspondig arrays to create a data dictionary baed on pairing of elements of the arrays with the same index. This data dictionary will be used to generate a heterogeneous graph. In the previous blogs I defined heterogeneous graphs as a graph, which contains either multiple types of objects or multiple types of links. For more information, please refer to my [introduction to graph theory](https://github.com/cyrusmvahid/Graph-BuildOn/blob/main/02-graph-theory.ipynb).

The eventual data dictionary will include: 1) user self loops, 2) item self loops, 3) user-item relationships, and 4) item-user relationships. The important part to focus on is the relations that start from user and interact with items and start with items and interact with a user. This is the part and we use to capture explicit collaboration signal that was lost in previous reincarnations of collaborative filtering.

```python
data_dict = {
            ...
            ("user", "ui", "item"): (user_item_src, user_item_dst),
            ("item", "iu", "user"): (user_item_dst, user_item_src),
        }
```

The only part that is left is to create a graph and we can begin the process of propogation and create necessary embedding.

```python
self.g = dgl.heterograph(data_dict, ...)
```

As you can see, graph *g* is a member variable of class *Data*.

### Embedding
